# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/plagiarism'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

sagemaker-pytorch-2021-03-11-03-52-29-818/source/sourcedir.tar.gz
sagemaker-pytorch-2021-03-11-03-53-23-189/profiler-output/system/incremental/2021031103/1615434960.algo-1.json
sagemaker-pytorch-2021-03-11-03-53-23-189/profiler-output/system/incremental/2021031103/1615435020.algo-1.json
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler-output/profiler-report.html
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler-output/profiler-report.ipynb
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler-output/profiler-reports/BatchSize.json
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler-output/profiler-reports/CPUBottleneck.json
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler-output/profiler-reports/Dataloader.json
sagemaker-pytorch-2021-03-11-03-53-23-189/rule-output/ProfilerReport-1615434803/profiler

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

### Define and Train a Built-In Classification Model

In [5]:
# import LinearLearner
from sagemaker import LinearLearner

output_path = 's3://{}/{}'.format(bucket, prefix)

#instantiate LinearLerner
linear = LinearLearner(role=role,
                      instance_count=1,
                      instance_type='ml.c4.xlarge',
                      predictor_type='binary_classifier',
                      output_path=output_path,
                      sagemaker_session=sagemaker_session,
                      epochs=15)

In [6]:
input_data

's3://sagemaker-us-east-1-859058257224/sagemaker/plagiarism'

In [7]:
import os

train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_label = train_df['0']
train_features = train_df.drop('0', axis=1)

In [8]:
train_label

0     1
1     1
2     0
3     0
4     0
     ..
64    1
65    1
66    1
67    1
68    0
Name: 0, Length: 69, dtype: int64

In [9]:
train_features_matrix = train_features.values
train_label_matrix = train_label.values

train_features_np = train_features_matrix.astype('float32')
train_label_np = train_label_matrix.astype('float32')

formatted_train_data = linear.record_set(train_features_np, labels=train_label_np)

In [10]:
%%time
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-11 05:19:28 Starting - Starting the training job...
2021-03-11 05:19:52 Starting - Launching requested ML instancesProfilerReport-1615439968: InProgress
.........
2021-03-11 05:21:13 Starting - Preparing the instances for training.........
2021-03-11 05:22:54 Downloading - Downloading input data
2021-03-11 05:22:54 Training - Downloading the training image...
2021-03-11 05:23:24 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[03/11/2021 05:23:19 INFO 140511992293184] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u


2021-03-11 05:23:55 Completed - Training job completed
Training seconds: 53
Billable seconds: 53
CPU times: user 813 ms, sys: 54.7 ms, total: 867 ms
Wall time: 4min 43s


In [11]:
%%time
#deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 340 ms, sys: 28.5 ms, total: 368 ms
Wall time: 9min 32s


In [12]:
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))
test_label = test_df['1']
test_features = test_df.drop('1', axis=1)

test_features_matrix = test_features.values
test_label_matrix = test_label.values

test_features_np = test_features_matrix.astype('float32')
test_label_np = test_label_matrix.astype('float32')

In [13]:
# test one prediction
result = linear_predictor.predict(test_features_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 1.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.995642900466919
    }
  }
}
]


In [14]:
import numpy as np

# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.
    Return binary classification metrics.
    :param predictor: A prediction enpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all preformance metrics
    :return: A dictionary of performance metrics
    """
    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 1)]
    
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch])
                                for batch in prediction_batches])
    
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format("Recall:", recall))
        print("{:<11} {:.3f}".format("Precision:", precision))
        print("{:<11}{:.3f}".format("Accuracy:", accuracy))
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
           'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}
        

In [15]:
print("Metrics for simple, LinearLearner.\n")

metrics = evaluate(linear_predictor,
                  test_features_matrix.astype('float32'),
                  test_label,
                  verbose=True)

Metrics for simple, LinearLearner.

prediction (col)  0.0  1.0
actual (row)              
0                  10    0
1                   0   14

Recall:     1.000
Precision:  1.000
Accuracy:  1.000


In [16]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [17]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-11-05-25-21-143


In [18]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

# sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. 
from sklearn.externals import joblib
# Import joblib package directly
#import joblib

## TODO: Import any additional libraries you need to define a model
from sklearn.linear_model import LogisticRegression

# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
 

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [19]:

# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

logistic_regression = SKLearn(entry_point='train.py',
                      py_version='py3',
                      framework_version='0.20.0', 
                      source_dir='source_sklearn',
                      role=role,
                      instance_count=1,
                      instance_type='ml.m4.xlarge',
                      sagemaker_session=sagemaker_session,
                      hyperparameters={'max_iter': 4})

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [20]:
%%time

# Train your estimator on S3 training data

logistic_regression.fit({'train': input_data})

2021-03-11 05:35:24 Starting - Starting the training job...
2021-03-11 05:35:47 Starting - Launching requested ML instancesProfilerReport-1615440924: InProgress
.........
2021-03-11 05:37:08 Starting - Preparing the instances for training......
2021-03-11 05:38:13 Downloading - Downloading input data...
2021-03-11 05:38:55 Training - Training image download completed. Training in progress..2021-03-11 05:38:55,486 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2021-03-11 05:38:55,488 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-11 05:38:55,501 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-03-11 05:38:55,741 botocore.utils INFO     IMDS ENDPOINT: http://169.254.169.254/
2021-03-11 05:38:55,922 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-11 05:38:58,947 sagemaker-training-toolkit INFO     No GPUs detected (normal if n

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [21]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = logistic_regression.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


-------------------!CPU times: user 324 ms, sys: 30.2 ms, total: 355 ms
Wall time: 9min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [23]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [24]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)
print('\nConfusion Matrix: ')
confusion_matrix(test_y, test_y_preds)

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

Confusion Matrix: 


array([[10,  0],
       [ 0, 15]])

## Define and Train a Pytorch Model

In [28]:
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data
import torch.nn as nn

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_

            'input_features': args.input_features,
            'hidden_dim': args.hidden_dim,
            'output_dim': args.output_dim,
        }
        torch.save(model_info, f)
        
    ## --- End of your code  --- ##
    

	# Save the model parameters
    model_path = os.path.join(args.model_dir, 'model.pth')
    with open(model_path, 'wb') as f:
        torch.save(model.cpu().state_dict(), f)


In [31]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch',
                    role=role,
                    framework_version='1.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.p2.xlarge',
                    sagemaker_session=sagemaker_session,
                    hyperparameters={'input_features': 3, 
                                     'hidden_dim': 64,
                                     'output_dim': 1,
                                     'epochs': 100,
                                     'lr': 1e-3})

In [32]:
%%time

estimator.fit({'train': input_data})

2021-03-11 06:05:03 Starting - Starting the training job...
2021-03-11 06:05:27 Starting - Launching requested ML instancesProfilerReport-1615442703: InProgress
.........
2021-03-11 06:06:48 Starting - Preparing the instances for training.........
2021-03-11 06:08:29 Downloading - Downloading input data...
2021-03-11 06:08:49 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-03-11 06:09:16,212 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-03-11 06:09:16,238 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-03-11 06:09:16,239 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-03-11 06:09:16,537 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-03-11 06:09:16,538 sagemaker-containers INFO     Generating setup.cfg
20

Training seconds: 82
Billable seconds: 82
CPU times: user 638 ms, sys: 42.2 ms, total: 680 ms
Wall time: 4min 43s


In [34]:
from sagemaker.pytorch import PyTorchModel

binary_clf = PyTorchModel(entry_point='predict.py',
                          source_dir='source_pytorch',
                          role=role,
                          framework_version='1.0',
                          py_version='py3',
                          model_data=estimator.model_data)

In [41]:
%%time

predictor = binary_clf.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------!CPU times: user 310 ms, sys: 5.47 ms, total: 315 ms
Wall time: 7min 32s


In [42]:
predictor

In [43]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [49]:
import numpy as np

# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate_pytorch(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.
    Return binary classification metrics.
    :param predictor: A prediction enpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all preformance metrics
    :return: A dictionary of performance metrics
    """
    
    test_preds = np.squeeze(np.rint(predictor.predict(test_x))).astype(int)
    
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format("Recall:", recall))
        print("{:<11} {:.3f}".format("Precision:", precision))
        print("{:<11}{:.3f}".format("Accuracy:", accuracy))
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
           'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [50]:
print("Metrics for PyTorch Model.\n")

metrics = evaluate_pytorch(predictor,
                          test_x,
                          test_y.values,
                          verbose=True)

Metrics for PyTorch Model.

prediction (col)   0   1
actual (row)            
0                 10   0
1                  1  14

Recall:     0.933
Precision:  1.000
Accuracy:  0.960


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**: I decided to train and deploy all three models just to see the kind of performance each one would have. So far for the LinearLerner and Sklearn models, I didn't get any false negatives or false positives. I only got one false negative with the PyTorch model. For the most part the number of false negatives and false positives have been low. This might have to do with the test dataset having only 25 samples. With such a low number, it's possible that the models would get such a hight score on accuracy.

### Question 2: How did you decide on the type of model to use? 

**Answer**: I trained all three models: LinearLearner, Sklearn, and PyTorch. I got an accuracy score of 1.00 with LinearLearner and Sklearn, but I only got a 0.96 with the PyTorch Model. If I were to choose which model I would like to use, I would probably go with the Logistic Regression model that I built using Sklearn. I decided to go with this model because I'm more familiar with the Sklearn library.



----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [51]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [52]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'VJPF50GJW9FZ863X',
   'HostId': '60uYxgMeDZlr9s73+liPJptSiTuXQaXzcgG2SeFdzp3jjbUvz2fpqn6KenT7KolWXlkOKhBPP2s=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '60uYxgMeDZlr9s73+liPJptSiTuXQaXzcgG2SeFdzp3jjbUvz2fpqn6KenT7KolWXlkOKhBPP2s=',
    'x-amz-request-id': 'VJPF50GJW9FZ863X',
    'date': 'Thu, 11 Mar 2021 06:51:49 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-pytorch-2021-03-11-05-52-00-413/rule-output/ProfilerReport-1615441920/profiler-output/profiler-reports/IOBottleneck.json'},
   {'Key': 'sagemaker-pytorch-2021-03-11-04-57-29-009/rule-output/ProfilerReport-1615438649/profiler-output/profiler-reports/IOBottleneck.json'},
   {'Key': 'sagemaker/plagiarism/linear-learner-2021-03-11-05-19-28-295/rule-output/ProfilerReport-1615439968/profiler-output/profiler-reports/GPUMemoryIncrease.json'}

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!